In [ ]:
#Main goal: Predict the number of aftershocks for a main earthquake using a Multilayer Perceptron (MLP)
#neural network for regression.

import os
import pandas as pd
import sys

#personal functions to get files
sys.path.append('../scripts')

current_directory = os.getcwd()
current_directory = os.getcwd()
clusters_folder_path = os.path.join(current_directory,'../ressources/clusters')
clusters_filename = 'clusters_dataset.csv'
cluster_file_path = os.path.join(clusters_folder_path,clusters_filename)


#loading data clusters_dataset
clusters_df = pd.read_csv(cluster_file_path)
clusters_df.head()

In [ ]:
#we add a column that will indicate if the earthquake was a mainshock, a foreshock or an aftershock
#-1 for foreshock, 0 for mainshock; 1 for aftershock

clusters_df['shockType'] = pd.Series(dtype=float)
for index, row in clusters_df.iterrows():
    if row['delta_days'] == 0. :
        clusters_df.at[index,'shockType'] = 0
    elif row['delta_days'] > 0. : 
        clusters_df.at[index,'shockType'] = 1
    elif row['delta_days'] < 0. :
        clusters_df.at[index,'shockType'] = -1

clusters_df.head()

If we want to be able to predict the number of aftershocks we first need to be able to have this data in the dataset.
With the associated MC_lab(id mega cluster spatial), SC_lab(id sub cluster spatial) ST_lab(id spatio temporal cluster) we are able to identify the number of aftershocks related to the main earthquake shock in the dataset. 

In [ ]:
#create a column for the number of aftershocks for each main earthquake
clusters_df['aftershocks_for_main'] = clusters_df.groupby(['MC_lab', 'SC_lab', 'ST_lab'])['shockType'].transform(lambda x: (x == 1).sum())

#create a column for the number of foreshocks for each main earthquake
clusters_df['foreshocks_for_main'] = clusters_df.groupby(['MC_lab', 'SC_lab', 'ST_lab'])['shockType'].transform(lambda x: (x == -1).sum())

#create a column for the total number of shocks
clusters_df['total_shocks'] = clusters_df.groupby(['MC_lab', 'SC_lab', 'ST_lab'])['shockType'].transform('count')

#display and check the new dataframe
clusters_df.head(10)


In [ ]:
#create columns for minimum magnitude of aftershocks, maximum magnitude of foreshocks, and minimum magnitude of foreshocks
clusters_df['max_aftershock_mag'] = clusters_df[clusters_df['shockType'] == 1].groupby(['MC_lab', 'SC_lab', 'ST_lab'])['mag'].transform('max')
clusters_df['min_aftershock_mag'] = clusters_df[clusters_df['shockType'] == 1].groupby(['MC_lab', 'SC_lab', 'ST_lab'])['mag'].transform('min')
clusters_df['max_foreshock_mag'] = clusters_df[clusters_df['shockType'] == -1].groupby(['MC_lab', 'SC_lab', 'ST_lab'])['mag'].transform('max')
clusters_df['min_foreshock_mag'] = clusters_df[clusters_df['shockType'] == -1].groupby(['MC_lab', 'SC_lab', 'ST_lab'])['mag'].transform('min')

#replace nan values with 0 (because not all mainshocks have foreshocks or aftershocks)
clusters_df['max_aftershock_mag'] = clusters_df['max_aftershock_mag'].fillna(0)
clusters_df['min_aftershock_mag'] = clusters_df['min_aftershock_mag'].fillna(0)
clusters_df['max_foreshock_mag'] = clusters_df['max_foreshock_mag'].fillna(0)
clusters_df['min_foreshock_mag'] = clusters_df['min_foreshock_mag'].fillna(0)

#checking
clusters_df.head()

In [ ]:
#identify rows where 'shockType' is 0 (mainshocks), this process is to have the maximum and minimum magnitudes for aftershocks and foreshocks associated with their mainshock
mainshocks_mask = clusters_df['shockType'] == 0

#find the maximum and minimum magnitudes values for aftershocks and foreshocks
max_aftershock_mag_values = clusters_df[clusters_df['shockType'] == 1].groupby(['MC_lab', 'SC_lab', 'ST_lab'])['mag'].max()
min_aftershock_mag_values = clusters_df[clusters_df['shockType'] == 1].groupby(['MC_lab', 'SC_lab', 'ST_lab'])['mag'].min()
max_foreshock_mag_values = clusters_df[clusters_df['shockType'] == -1].groupby(['MC_lab', 'SC_lab', 'ST_lab'])['mag'].max()
min_foreshock_mag_values = clusters_df[clusters_df['shockType'] == -1].groupby(['MC_lab', 'SC_lab', 'ST_lab'])['mag'].min()

#change 'max_aftershock_mag', 'min_aftershock_mag', 'max_foreshock_mag' and 'min_foreshock_mag' columns on mainshock rows
clusters_df.loc[mainshocks_mask, 'max_aftershock_mag'] = clusters_df.loc[mainshocks_mask, ['MC_lab', 'SC_lab', 'ST_lab']].apply(
    lambda row: max_aftershock_mag_values.get(tuple(row), 0), axis=1
)
clusters_df.loc[mainshocks_mask, 'min_aftershock_mag'] = clusters_df.loc[mainshocks_mask, ['MC_lab', 'SC_lab', 'ST_lab']].apply(
    lambda row: min_aftershock_mag_values.get(tuple(row), 0), axis=1
)

clusters_df.loc[mainshocks_mask, 'max_foreshock_mag'] = clusters_df.loc[mainshocks_mask, ['MC_lab', 'SC_lab', 'ST_lab']].apply(
    lambda row: max_foreshock_mag_values.get(tuple(row), 0), axis=1
)
clusters_df.loc[mainshocks_mask, 'min_foreshock_mag'] = clusters_df.loc[mainshocks_mask, ['MC_lab', 'SC_lab', 'ST_lab']].apply(
    lambda row: min_foreshock_mag_values.get(tuple(row), 0), axis=1
)

clusters_df.head()

In [ ]:
clusters_df.info()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler


#only using mainshock data rows
mainshocks_df = clusters_df[clusters_df['shockType'] == 0]

#features and target variable
features = mainshocks_df[['depth','mag', 'max_foreshock_mag', 'min_foreshock_mag', 'foreshocks_for_main']]
target = mainshocks_df['aftershocks_for_main']

#spliting data into training and testing sets (20%)
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

#standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

#initialize MLPRegressor, 2 hidden layers(100 and 50 neurones)
mlp_regressor = MLPRegressor(hidden_layer_sizes=(100, 50), max_iter=500, random_state=42)

#train model
mlp_regressor.fit(X_train_scaled, y_train)

#predictions on the test set
predictions = mlp_regressor.predict(X_test_scaled)

#evaluate model
mse = mean_squared_error(y_test, predictions)
print('Mean Squared Error on Test Set: ', mse)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

#scatter plot of actual vs predicted values for every main earthquake(points)
plt.figure(figsize=(10, 6))

#scatter plot with fitting line of points
sns.regplot(x=y_test, y=predictions, line_kws={'color': 'red'})

#ideal diagonal line (x=y)
plt.plot([0, max(y_test)], [0, max(y_test)], linestyle='--', color='blue', label='x=y')

plt.title('Actual vs Predicted Aftershocks')
plt.xlabel('Actual Aftershocks')
plt.ylabel('Predicted Aftershocks')
plt.legend()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

#dataframe for actual and predicted values
comparison_df = pd.DataFrame({'Actual': y_test, 'Predicted': predictions})

#group by mainshock and sum the aftershocks
mainshock_comparison = comparison_df.groupby(comparison_df.index).agg({'Actual': 'sum', 'Predicted': 'sum'})

#plotting bar chart
plt.figure(figsize=(12, 8))
ax = mainshock_comparison.plot(kind='bar', stacked=True)
ax.set_xticks([])  # Remove x-axis ticks
ax.set_title('Actual vs Predicted Aftershocks for Each Mainshock')
ax.set_xlabel('Mainshock Index')
ax.set_ylabel('Number of Aftershocks')
plt.show()


In [ ]:
from sklearn.model_selection import GridSearchCV

#still looking at only the mainshock rows
mainshocks_df = clusters_df[clusters_df['shockType'] == 0]

#selecting features and target variable
features = mainshocks_df[['depth','mag','max_aftershock_mag', 'min_aftershock_mag', 'max_foreshock_mag', 'min_foreshock_mag']]
target = mainshocks_df['aftershocks_for_main']

#plit data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

#standardize feature
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

#define the parameter grid to search
#param_grid = {
#    'hidden_layer_sizes': [(50,), (100,), (50, 25), (100, 50), (100, 100, 50)],
#    'activation': ['relu', 'tanh'],
#    'solver': ['adam', 'sgd'],
#    'learning_rate': ['constant', 'invscaling', 'adaptive'],
#    'alpha': [0.0001, 0.001, 0.01],
#    'max_iter': [200, 300, 400],
#    'early_stopping': [True, False],
#}

#Results(1 hour of computation):
#Best Hyperparameters: {'activation': 'relu', 'alpha': 0.01, 'early_stopping': False, 'hidden_layer_sizes': (100, 50), 'learning_rate': 'constant', 'max_iter': 400, 'solver': 'adam'}
#Mean Squared Error on Test Set: 1047.7739561961562

#less grid parameters for 1 min computation
param_grid = {
    'hidden_layer_sizes': [(2**6, 2**7, 2**6), (2**7, 2**6)],
    'activation': ['relu', 'tanh', 'logistic'],
    'solver': ['adam'],
    'learning_rate': ['constant'],
    'alpha': [0.01],
    'max_iter': [400],
}

mlp_regressor = MLPRegressor(random_state=42)

#GridSearchCV
grid_search = GridSearchCV(mlp_regressor, param_grid, cv=3, scoring='neg_mean_squared_error', n_jobs=-1)

#fit model
grid_search.fit(X_train_scaled, y_train)

#obtain best parameters
best_params = grid_search.best_params_
print(f'Best Hyperparameters: {best_params}')

#prediction on the test set
predictions = grid_search.predict(X_test_scaled)

#evaluate model
mse = mean_squared_error(y_test, predictions)
print('Mean Squared Error on Test Set: ', mse)
